In [30]:
import sys
import os

In [31]:
%pwd

'D:\\MLOPS\\datascienceproject'

In [32]:
import os

# Change directory to your project folder
os.chdir(r"D:\MLOPS\datascienceproject")

# Verify the change
print("Updated Working Directory:", os.getcwd())


Updated Working Directory: D:\MLOPS\datascienceproject


In [33]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "src")))


In [34]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [35]:
from src.Datascience.constants import *
from src.Datascience.utils.common import read_yaml, create_directories
from src.Datascience.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH

In [36]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        """Initialize ConfigurationManager and read YAML configurations."""
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
 
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [37]:
import requests
import urllib.request as request 
from src.Datascience import logger
import zipfile

In [38]:
##Component-Data Ingestion
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    #Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file  
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

            #extract zip file
    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file into 
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
               

In [39]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)  
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e  

[2025-02-22 16:18:53,283: INFO: common: YAML file config\config.yaml loaded successfully]
[2025-02-22 16:18:53,369: INFO: common: YAML file params.yaml loaded successfully]
[2025-02-22 16:18:53,371: INFO: common: YAML file schema.yaml loaded successfully]
[2025-02-22 16:18:53,372: INFO: common: Created directory at: artifacts]
[2025-02-22 16:18:53,374: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-02-22 16:18:53,378: INFO: 2248113168: File already exists]
